<h1><center>Laboratorio 7: La desperación de Mr. Lepin 🐼</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Ignacio Meza, Gabriel Iturra
- Auxiliar: Sebastián Tinoco
- Ayudante: Arturo Lazcano, Angel Muñoz

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Jorge Allendes
- Valentina Castro


### **Link de repositorio de GitHub:** `https://github.com/J07Aa/Repo`

### Indice

1. [Temas a tratar](#Temas-a-tratar:)
3. [Descripción del laboratorio](#Descripción-del-laboratorio.)
4. [Desarrollo](#Desarrollo)

# Temas a tratar

- Aplicar Pandas para obtener características de un DataFrame.
- Aplicar Pipelines.
- Aplicar Clusters sobre un conjunto de datos.

## Reglas:

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias.
- Pueden usar cualquer matrial del curso que estimen conveniente.
- Código que no se pueda ejecutar, no será revisado.

### Objetivos principales del laboratorio

- Comprender y aprovechar las ventajas que nos ofrece la librería `pandas` con respecto a trabajar en Python 'puro'.
- Crear nuevas características para entrenar un modelo de clustering.
- Comprender como aplicar pipelines de Scikit-Learn para generar procesos más limpios.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `numpy`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre arreglos (*o tensores*).

## Descripción del laboratorio.

### Importamos librerias utiles 😸

In [1]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
import datetime
from IPython.display import HTML

# Libreria para plotear (En colab esta desactualizado plotly)
!pip install --upgrade plotly
import plotly.express as px
import plotly.graph_objects as go

# Librerias utiles
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.6/15.6 MB 71.1 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.15.0
    Uninstalling plotly-5.15.0:
      Successfully uninstalled plotly-5.15.0


# Segmentación de Clientes en Tienda de Retail 🛍️

<p align="center">
  <img width=300 src="https://s1.eestatic.com/2018/04/14/social/la_jungla_-_social_299733421_73842361_854x640.jpg">
</p>

## 1.1 Cargar Dataset

Mr. Lepin, en una nueva reunión, le cuenta a ud y su equipo que los resultados derivados del análisis exploratorio de dato presentaron una gran utilidad para la empresa y que tiene un gran entusiasmo por continuar trabajando con ustedes.
Es por esto, que Mr. Lepin les pide que cargue y visualicen algunas de las filas que componen el Dataset.
A continuación un extracto de lo parlamentado en la reunión:

    - Usted: Es un gran logro para nuestro equipo que usted haya encontrado excelente el EDA. ¿Qué tiene en mente ahora?
    - Mr. Lepin: Resulta que hace algún tiempo, mientras tomaba un mojito en una reunión de gerentes en Panamá, oí a un *chato* acerca de **LRMFP**, que es un modelo que permite personificar a los clientes a través de la fabricación de distintos atributos que describen a los clientes. Lo encontré es-tu-pendo ñatito.
    - Usted: Ehh bueno. Investigaremos acerca de este modelo y veremos lo que podemos hacer.

Por ende, su siguiente tarea es calcular **LRMFP** sobre cada cliente y luego hacer un análisis de las características generadas. Para esto, el área de ventas les entrega un nuevo archivo llamado `online_retail_II_cleaned.pickle`, quien posee los datos del DataFrame original limpios y listos para obtener las características solicitadas por Mr. Lepin.

In [2]:
df_retail = pd.read_pickle("online_retail_II_cleaned.pickle")
df_retail = df_retail.astype(
    {
        "Invoice": "category",
        "StockCode": "category",
        "Description": "category",
        "Description": str,
        "Customer ID": "category",
        "Country": "category"
    }
)
df_retail.head()


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [3]:
df_retail.describe()

,Quantity,Price
count,400947.000000,400947.000000
mean,13.768523,3.305571
std,97.639816,35.046376
min,1.000000,0.000000
25%,2.000000,1.250000
50%,5.000000,1.950000
75%,12.000000,3.750000
max,19152.000000,10953.500000


## 1.2 Creación de nuevas Caracteristicas [2 Puntos]

Como ya se les comento, Mr. Lepin esta interesado en obtener las características **LRMFP**, para esto les señala que estas características se construyen en base a las siguientes definiciones:

- **Length (L)**: Intervalo de tiempo, en días, entre la primera y la última visita del cliente. Mientras mas grande sea el valor, mas fiel es el cliente.

- **Recency (R)**: Indica hace cuanto tiempo el cliente realizo su ultima compra. Notar que para este caso, mientras mas grande es el valor, menos interes posee el usuario para repetir una compra en uno de los locales.

- **Monetary (M)**: El término "monetario" se refiere a la cantidad media de dinero gastada por cada visita del cliente durante el período de observación y refleja la contribución del cliente a los ingresos de la empresa.

- **Frequency (F)**: Se refiere al número total de visitas del cliente durante el periodo de observación. Cuanto mayor sea la frecuencia, mayor será la fidelidad del cliente.

- **Periodicity (P)**: Representa si los clientes visitan las tiendas con regularidad.

$$Periodicity(n)=std(IVT_1, ..., IVT_n)$$

&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Donde $IVT$ denota el tiempo entre visitas y n representa el número de valores de tiempo entre visitas de un cliente.


$$IVT_i=date\_diff(t_{i+1},t)$$

En base a las definiciones señaladas, diseñe una función que permita obtener las características **LRMFP** recibiendo un DataFrame como entrada. Para esto, no estará permitido el uso de iteradores, utilice todas las herramientas que les ofrece `pandas` para realizar esto.

Una referencia que le puede ser útil es el [documento original](https://www.researchgate.net/publication/315979555_LRFMP_model_for_customer_segmentation_in_the_grocery_retail_industry_a_case_study) en donde se propone este método.

**<u>Formato</u> del Resultado Esperado:**

| Customer ID | Length | Recency | Frequency | Monetary | Periodicity |
|------------:|-------:|--------:|----------:|---------:|------------:|
|   12346.0   |    294 |      67 |        46 |   -64.68 |        37.0 |
|   12347.0   |     37 |       3 |        71 |  1323.32 |         0.0 |
|   12349.0   |    327 |      43 |       107 |  2646.99 |        78.0 |
|   12352.0   |     16 |      11 |        18 |   343.80 |         0.0 |
|   12356.0   |     44 |      16 |        84 |  3562.25 |        12.0 |

**Respuesta:**

In [8]:
def custom_features(dataframe_in):

  dataframe_in = df_retail
  df_retail['Ingreso'] = df_retail['Price'] * df_retail['Quantity']
  df_retail['InvoiceDate'] = pd.to_datetime(df_retail['InvoiceDate'], dayfirst=True)
  df_retail['InvoiceDate']=df_retail['InvoiceDate'].dt.strftime('%Y-%m-%d')
  df_retail['InvoiceDate'] = pd.to_datetime(df_retail['InvoiceDate'], dayfirst=True)

  ## Agrupamos por cliente y obtenemos el minimo y máximo de las fechas de compra
  ## para calcular L y R
  df_aux_1 = df_retail.groupby(['Customer ID']).agg({"InvoiceDate":['min','max']})
  # Se calcula L como la fecha más nueva menos la más antigua
  df_aux_1["L"]= df_aux_1[("InvoiceDate",'max')] - df_aux_1[("InvoiceDate",'min')]

  # Se calcula R tomando como fecha actual la fecha más grande, más un día
  df_aux_1["R"] =  max(df_aux_1[("InvoiceDate",'max')]) + datetime.timedelta(days=1) - df_aux_1[("InvoiceDate",'max')]
  ## Agrupamos por cliente y fecha sumando los ingresos, asumiendo que por cada día
  ## solo puede existir una visita y que los ingresos es igual al precio por la cantidad
  df_aux_2 = df_retail.groupby(['Customer ID','InvoiceDate'],).sum(numeric_only = True).round(2)

  ##Se eliminan los ingresos iguales a 0 (son indices adicionales agregados por gruoupby)
  df_aux_2 = df_aux_2[df_aux_2['Quantity']!=0]

  ## Se resetea el indice luego de eliminar los ingreso 0
  df_aux_2 = df_aux_2.reset_index()

  ## Se vuelve a agrupar con el fin de calcular el promedio de compras diario y la cantidad de compras.
  df_aux_2 = df_aux_2.groupby(['Customer ID']).agg({"Ingreso":['mean','count']})
  display(df_aux_2.columns)
  df_aux_2.columns = ['M','F']

  df_IVT = df_retail.groupby(['Customer ID','InvoiceDate']).agg({"InvoiceDate":['count']})
  df_IVT = df_IVT[df_IVT[('InvoiceDate','count')]!=0]
  df_IVT = df_IVT.reset_index()
  df_IVT_2 = pd.DataFrame()
  df_IVT_2['Customer ID'] = df_IVT[('Customer ID',      '')]
  df_IVT_2['InvoiceDate'] = df_IVT[('InvoiceDate',      '')]
  df_IVT_2['IVT'] = df_IVT_2.groupby(['Customer ID'])['InvoiceDate'].diff()
  df_IVT_2 = df_IVT_2.dropna()
  df_IVT_2['IVT'] = df_IVT_2['IVT'].dt.days.astype('int')
  df_IVT_2 = df_IVT_2.groupby(['Customer ID'])['IVT'].std()
  d = pd.DataFrame(df_IVT_2).reset_index()
  df_IVT_3 = pd.DataFrame()
  #display(d.columns)
  df_IVT_3['Customer ID'] = d['Customer ID']
  df_IVT_3['P'] = d['IVT']
  df_IVT_3 = df_IVT_3.fillna(0)
  df_IVT_3 = df_IVT_3.set_index(['Customer ID'])
  # Ya con todo calculado se procede a crear el dataframe final como:
  df = df_aux_1[['L','R']]
  df.columns = ['L','R']
  df['L'] = df['L'].dt.days.astype('int')
  df['R'] = df['R'].dt.days.astype('int')
  df = df.join(df_aux_2)
  df = df.join(df_IVT_3)
  return df


In [9]:
df = custom_features(df_retail)
df.head()

MultiIndex([('Ingreso',  'mean'),
            ('Ingreso', 'count')],
           )

<ipython-input-8-7fd75dac3102>:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['L'] = df['L'].dt.days.astype('int')
<ipython-input-8-7fd75dac3102>:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['R'] = df['R'].dt.days.astype('int')


,L,R,M,F,P
Customer ID,,,,,
12346.0,196,165,53.265714,7,43.614982
12347.0,37,3,661.660000,2,0.000000
12348.0,0,74,222.160000,1,0.000000
12349.0,182,43,890.380000,3,101.823376
12351.0,0,11,300.930000,1,0.000000


**Resúesta de Pauta:**

## 1.3 Pipelines 👷

Finalmente *Don Mora* le pregunta si seria posible realizar un pipeline para realizar una segmentación de los clientes con los nuevos datos generados, a lo que usted responde que **sí** y propone la utilización de k-means para la segmentación.

A continuación siga los pasos requeridos para obtener la segmentación de clientes.

### 1.3.1 Estandarizar Caracteristicas [0.5 puntos]

Construya una clase llamada ``MinMax()`` utilizando ``BaseEstimator`` y ``TransformerMixin`` para realizar una transformación de cada una de las columnas de un DataFrame utilizando ``ColumnTransformer()`` más tarde (tome como referencia el siguiente [enlace](https://sklearn-template.readthedocs.io/en/latest/user_guide.html#transformer)).


 Para esto considere que Min-Max escaler queda dada por la ecuación:

$$MinMax = \dfrac{x-min(x)}{max(x) - min(x)}$$

Con esto buscamos que los valores que componen a las columnas se muevan en el rango de valores $[0, 1]$.

**Respuesta:**

In [10]:
# Creamos la clase
class MinMax(BaseEstimator, TransformerMixin):
    # Primero definiremos valores mínimo y máximo para guardarlos.
    def __init__(self):
        self.c_min = 0
        self.c_max = 0

    def fit(self, X, y=None):
        # Calculamos los mínimos y máximos para cada columna
        self.c_max = X.min()
        self.c_min = X.max()
        return self

    def transform(self, X):
        # Aplicamos la transformación utilizando los valores calculados en fit
        X_norm = (X - self.c_min) / (self.c_max - self.c_min)
        return X_norm


In [11]:
# generamos el pipeline
Sacler_pipe = Pipeline([('scaler', MinMax())
                        ])
preprocessing_transformer = ColumnTransformer(
    transformers=[
        ('min_max_transform', MinMax(),  ['L', 'R', 'M', 'F', 'P']),
        ])
retail_pipeline = Pipeline([
    ('Preprocessing', preprocessing_transformer)
])

In [15]:
#Estalamos el dataframe
df_preprocesado = retail_pipeline.fit_transform(df)
df_preprocesado = pd.DataFrame(df_preprocesado)
df_preprocesado

,0,1,2,3,4
0,0.474531,0.560322,0.996172,0.952756,0.829611
1,0.900804,0.994638,0.952454,0.992126,1.000000
2,1.000000,0.804290,0.984036,1.000000,1.000000
3,0.512064,0.887399,0.936019,0.984252,0.602210
4,1.000000,0.973190,0.978376,1.000000,1.000000
...,...,...,...,...,...
4309,0.260054,0.954424,0.991099,0.968504,0.757976
4310,1.000000,0.823056,0.966825,1.000000,1.000000
4311,1.000000,0.209115,0.969317,1.000000,1.000000
4312,0.337802,0.702413,0.953421,0.992126,1.000000


### 1.3.2 `T-SNE` Pipeline [1.0 puntos]

Para comenzar introduciéndose en el uso de pipeline, decide probar realizando un pipeline enfocado en la reducción de dimensionalidad y así hacer no decepcionar a Mr. Lepin con la clusterización del modelo.

Configure un pipeline utilizando el algoritmo `T-SNE` sobre los datos **LRMFP**, donde, para la realización del pipeline considera los siguientes pasos:

1. Como primer paso obtenga las características **LRMFP** desde el DataFrame ``df_retail_II_cleaned.pickle`` utilizando la función ``custom_features`` creada anteriormente, junto a ``FunctionTransformer()``. Considere esto como el primer paso de su pipeline.
2. En segundo lugar usando ``ColumnTransformer()`` aplique el MinxMax scaler creado por usted sobre todas las columnas generadas en el paso anterior.
3. Finalmente, aplique un último paso donde obtiene las 2 componentes más relevantes utilizando el algoritmo `T-SNE` de sckit-learn.

Tras aplicar las transformaciones sobre el dataset **LRMFP**, gráfique las componentes obtenidas en la reducción de dimensionalidad.

**Respuesta:**

In [19]:
# Paso 1
transformer = FunctionTransformer(custom_features)

In [20]:
# Paso 2
# generamos el pipeline
Sacler_pipe = Pipeline([('scaler', MinMax())
                        ])
preprocessing_transformer = ColumnTransformer(
    transformers=[
        ('min_max_transform', MinMax(),  ['L', 'R', 'M', 'F', 'P']),
        ])

In [21]:
# Paso 3
tsne = TSNE(n_components=2)

In [22]:
retail_pipeline = Pipeline([
    ('custom_features',transformer),
    ('Preprocessing', preprocessing_transformer),
    ('main_features',tsne)
])

In [23]:
df_new = retail_pipeline.fit_transform(df_retail)

MultiIndex([('Ingreso',  'mean'),
            ('Ingreso', 'count')],
           )

<ipython-input-8-7fd75dac3102>:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['L'] = df['L'].dt.days.astype('int')
<ipython-input-8-7fd75dac3102>:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['R'] = df['R'].dt.days.astype('int')


In [29]:
df_new = pd.DataFrame(df_new)
df_new.head()

,0,1
0,-7.834351,17.751034
1,-0.094690,-37.516727
2,-45.633453,-38.774620
3,17.810129,24.241663
4,9.276519,-56.333603


In [32]:
#Ahora procedemos a plotear los resultados

fig = px.scatter(df_new, x=1,y=0)
fig.show()

### 1.3.3 Clustering

#### 1.3.3.1 Método del Codo [1 puntos]

Utilizando la clase creada para escalamiento, aplique el método del codo para visualizar cual es el número de clusters que mejor se ajustan a los datos. Realice esto utilizando el algoritmo K-means dentro de un pipeline para un $k \in [1,20]$, donde k representa el número de clusters del k-means. Para la realización de esta sección y la próxima (1.3.3.2), considere los mismos pasos utilizados para el t-sne, pero **permutando el algoritmo de reducción de dimensionalidad por k-means.**

A través del grafico obtenido, comente y justifique que valor de k escogería para realizar el k-means.

**Respuesta:**

In [34]:
elbow_scores = []

for k in range(1, 21):
    pipeline = Pipeline([
    ('custom_features',transformer),
    ('Preprocessing', preprocessing_transformer),
    ('kmeans-k',KMeans(n_clusters=k))
    ])

    # Ajustar el modelo
    pipeline.fit(df_retail)

    # Calcular la puntuación de inercia (within-cluster sum of squares)
    elbow_scores.append(pipeline.named_steps['kmeans-k'].inertia_)

# Graficar los resultados del método del codo
fig = px.line(x=range(1, 21), y=elbow_scores, markers=True, line_shape='linear')
fig.update_traces(mode='lines+markers')
fig.update_layout(
    xaxis_title='Número de Clusters (k)',
    yaxis_title='Inercia',
    title='Método del Codo para Encontrar el Mejor k'
)
fig.show()

MultiIndex([('Ingreso',  'mean'),
            ('Ingreso', 'count')],
           )

<ipython-input-8-7fd75dac3102>:52: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-8-7fd75dac3102>:53: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



MultiIndex([('Ingreso',  'mean'),
            ('Ingreso', 'count')],
           )

<ipython-input-8-7fd75dac3102>:52: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-8-7fd75dac3102>:53: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



MultiIndex([('Ingreso',  'mean'),
            ('Ingreso', 'count')],
           )

...

<ipython-input-8-7fd75dac3102>:52: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-8-7fd75dac3102>:53: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



MultiIndex([('Ingreso',  'mean'),
            ('Ingreso', 'count')],
           )

<ipython-input-8-7fd75dac3102>:52: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-8-7fd75dac3102>:53: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



MultiIndex([('Ingreso',  'mean'),
            ('Ingreso', 'count')],
           )

<ipython-input-8-7fd75dac3102>:52: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-8-7fd75dac3102>:53: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



Luego de obtener el gráfico de Inercia de K-Means, para los valores de k pedidos, llegamos a la conclusión. que el codo se encuentra entre los valores k=3,4,5 y decidimos quedarnos con K=5 ya que es el último momento en que la inercia varía significativamente.

#### 1.3.3.2 Segmentación de Clientes con K-Means 🎁 [1 punto]

En base a la elección de k realizada en la sección anterior, utilice este valor escogido y entrene un modelo de K-means utilizando el mismo pipeline de scikit-learn utilizado anteriormente.

Una vez ajustado los datos, genere una tabla con los promedios (o medianas) para cada uno de los atributos, agrupando estos por el clúster que pertenecen. ¿Es posible observar agrupaciones coherentes?, ¿Qué tipo de clientes posee el retail?, Justifique su respuesta y no decepcione a Mr. Lepin.


**Respuesta:**

In [37]:
pipeline = Pipeline([
    ('custom_features',transformer),
    ('Preprocessing', preprocessing_transformer),
    ('kmeans-5',KMeans(n_clusters=5, random_state=21))
    ])

# Ajustar el modelo
pipeline.fit(df_retail)

# Obtener los clusters
df_labels = pipeline.named_steps['kmeans-5'].labels_

df = transformer.transform(df_retail)
df['labels'] = df_labels
df.head()

MultiIndex([('Ingreso',  'mean'),
            ('Ingreso', 'count')],
           )

<ipython-input-8-7fd75dac3102>:52: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-8-7fd75dac3102>:53: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



MultiIndex([('Ingreso',  'mean'),
            ('Ingreso', 'count')],
           )

<ipython-input-8-7fd75dac3102>:52: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-8-7fd75dac3102>:53: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,L,R,M,F,P,labels
Customer ID,,,,,,
12346.0,196,165,53.265714,7,43.614982,2
12347.0,37,3,661.660000,2,0.000000,0
12348.0,0,74,222.160000,1,0.000000,0
12349.0,182,43,890.380000,3,101.823376,1
12351.0,0,11,300.930000,1,0.000000,0


In [38]:
# ahora se augrupa por cluster y obtenemos el valor promedio de los indicadores

df = df.groupby('labels').mean()
df.head()

,L,R,M,F,P
labels,,,,,
0,16.786109,52.074980,406.037826,1.565114,1.465408
1,283.818482,34.729373,385.311339,3.838284,107.151225
2,177.261803,69.247854,396.344667,3.919528,22.370870
3,17.467547,255.536854,384.683100,1.372937,1.657000
4,322.099668,22.606866,489.037347,9.915836,27.071413


Tipos de Clientes:
- Tipo 0:
  - Clientes poco fieles ('L' pequeño)
  - Medianemente interesado volver a comprar ('R' mediano)
  - Todos realizan un aporte monetario similar
  - Clientes poco fieles ('F' pequeño)
  - Persona que suelen asistir entre ventanas cortas de tiempo ('P' pequeño)
- Tipo 1:
  - Clientes fieles ('L' grande)
  - Interesado en volver a comprar ('R' pequeño)
  - Todos realizan un aporte monetario similar
  - Cliente fiel ('F' Mediano)
  - Persona que se toma un buen descanso entre cada visita a la tienda ('P' grande)
- Tipo 2:
  - Clientes en proceso de fidelización ('L' mediano)
  - Medianemente interesado volver a comprar ('R' mediano)
  - Todos realizan un aporte monetario similar
  - Cliente fiel ('F' Mediano)
  - Personas que asisten periodicamente a la tienda ('P' mediano)
- Tipo 3:
  - Clientes poco fieles ('L' pequeño)
  - Poco o nada interesado en volver a comprar ('R' grande)
  - Todos realizan un aporte monetario similar
  - Clientes poco fieles ('F' pequeño)
  - Persona que suelen asistir entre ventanas cortas de tiempo ('P' pequeño)
- Tipo 4:
  - Clientes fieles ('L' grande)
  - Interesado en volver a comprar ('R' pequeño)
  - Todos realizan un aporte monetario similar
  - Cliente muy fiel ('F' grandes)
  - Personas que asisten periodicamente a la tienda ('P' mediano)

Entonces el primer cliente (tipo 0) debido a 'R', se podría considerar un cliente nuevo que hizo un par de visitas seguisdas (por 'F' y 'P') y es poco fiel ('L').

Es segundo tipo (tipo 1) es un cliente fiel ('L'), interesado en volver a comprar ('R'), que visita constantemenete la tienda ('F'), pero el tiempo entre visitas es largo ('P').

El cliente tipo 2, está en proceso de ser fiel ('L'), no está muy interesado en volver a comprar ('R'), pero suele asistir regularmente a la tienda ('F') y su periodo entre visitas es corto ('P').

Para el cliente tipo 3, se tiene que es un cliente que hace mucho no va a la tienda ('R'), es poco fiel ('L'), no fué mucho a la tienda ('F'), pero fué seguido ('P'). Probablemente en su momento necesitaron cosas de la tienda, pero ya no han necesitado ir a la tienda o no les gustó.

Finalmente el cliente tipo 4 es muy fiel ('L') y debería estar interesado en volver a comprar ('R'), suele ser un cliente recurrente ('F'), pero el tiempo entre sus visitas es largo ('P').

No hablamos de lo monetario ya que es similar para casi todos los clientes, solo el cliente tipo 4 suele gastar un poco más en promedio.

**Respuesta Esperada:**

|         | Length  | Recency   | Frequency | Monetary | Periodicity |       |
|---------|---------|-----------|----------|-------------|-------|-------|
| Cluster |         |           |          |             |       |       |
|    0    |   258.8 |      45.2 |     76.1 |      1107.7 | 107.6 |   449 |
|    1    |    76.1 |     217.6 |     45.5 |       791.7 |  14.1 |   466 |
|    2    |   368.5 |       4.8 |   2715.0 |    226621.6 |   4.2 |     4 |
|    3    |    85.3 |      45.7 |     65.8 |      1047.0 |  10.5 |   987 |
|    4    |   347.2 |      15.9 |   1658.0 |     35829.3 |   8.0 |    25 |
|    5    |   298.0 |      29.8 |    183.8 |      3639.9 |  32.0 |  1188 |

#### 1.3.3.3 Plot de K-Means 📈 [0.5 puntos]



Por último, Mr. Lepin, impaciente de no entender lo que usted intenta explicarle, le solicita que por favor muestre algún resultado "visual" de los grupos encontrados.

Para esto, grafique nuevamente las características encontradas usando `T-SNE` (no calcule de nuevo, simplemente utilice las proyecciones encontradas) y agregue las labels calculadas con kmeans como el argumento `color`.   

Comente: ¿Se separan bien los distintos clusters en la visualización?


**Respuesta:**

In [39]:
#Ahora procedemos a plotear los resultados de TSNE con los labels de K-Means

fig = px.scatter(df_new, x=1,y=0, color = df_labels)
fig.show()

De la gráfica anterior se observa que los grupos están bastante bien definidos,a pesar de existir solapamiento en las fronterras.
Por lo que podríamos asegurar que hay 5 tipos de clientes que han visitado la tienda.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por correo, Discord o U-cursos.

![Gracias Totales!](https://i.pinimg.com/originals/65/ae/27/65ae270df87c3c4adcea997e48f60852.gif "bruno")


<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>